# Assignment 4 — SEMMA Case Study (Adult Income Classification)

Dataset: [`scikit-learn/adult-census-income`](https://huggingface.co/datasets/scikit-learn/adult-census-income)

Goal: Predict income (<=50K vs >50K) using SEMMA: Sample, Explore, Modify, Model, Assess.

Environment: Python 3.11 (`.venv`), scikit-learn, imbalanced-learn, pandas, seaborn, SHAP.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from datasets import load_dataset
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import shap

sns.set_theme(style="whitegrid")
FIG_DIR = Path("figures")
FIG_DIR.mkdir(exist_ok=True)
RNG = 42
pd.set_option("display.max_columns", 50)

## Sample
- Load dataset from Hugging Face
- Stratified split train/valid/test

In [ ]:
ds = load_dataset("scikit-learn/adult-census-income", split="train")
df = ds.to_pandas()
df.shape, df["income"].value_counts(normalize=True)

In [ ]:
target = "income"
X = df.drop(columns=[target])
y = df[target].apply(lambda v: 1 if v.strip() == ">50K" else 0)

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RNG
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=RNG
)
(X_train.shape, X_valid.shape, X_test.shape)

## Explore
- Missingness, target balance
- Univariate distributions (numeric)
- Categorical frequency
- Correlations (numeric) and target relationship

In [ ]:
missing = df.isna().sum().sort_values(ascending=False)
missing.head(10)

In [ ]:
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object", "string"]).columns.tolist()

fig, axes = plt.subplots(2, len(numeric_cols)//2, figsize=(14, 6))
axes = axes.flatten()
for ax, col in zip(axes, numeric_cols):
    sns.histplot(df[col], kde=True, ax=ax)
    ax.set_title(col)
plt.tight_layout()
univar_num_fig = FIG_DIR / "univar_numeric.png"
plt.savefig(univar_num_fig)
univar_num_fig

In [ ]:
plt.figure(figsize=(10,6))
freq = df[target].value_counts(normalize=True) * 100
sns.barplot(x=freq.index, y=freq.values)
plt.title("Target balance (% >50K vs <=50K)")
plt.tight_layout()
balance_fig = FIG_DIR / "target_balance.png"
plt.savefig(balance_fig)
balance_fig

In [ ]:
corr = df[numeric_cols + [target]].assign(income=y).corr()["income"].drop("income")
top_corr = corr.abs().sort_values(ascending=False).head(10).index
plt.figure(figsize=(8,6))
sns.heatmap(df[list(top_corr) + [target]].assign(income=y).corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation with income (top numeric)")
plt.tight_layout()
corr_fig = FIG_DIR / "corr_top_numeric.png"
plt.savefig(corr_fig)
corr_fig

## Modify
- Impute + one-hot; scale numerics for linear models
- Compare class_weight vs SMOTE for imbalance
- Simple feature tweaks: bin age, combine rare categories if needed (skipped here for brevity)

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols),
])

def evaluate_clf(pipe, Xtr, ytr, Xval, yval, name: str):
    pipe.fit(Xtr, ytr)
    proba = pipe.predict_proba(Xval)[:, 1]
    preds = (proba >= 0.5).astype(int)
    roc = roc_auc_score(yval, proba)
    pr = average_precision_score(yval, proba)
    return {"name": name, "roc_auc": roc, "avg_precision": pr, "pipeline": pipe}

results = []
results.append(evaluate_clf(
    Pipeline([
        ("prep", preprocessor),
        ("model", DummyClassifier(strategy="most_frequent"))
    ]),
    X_train, y_train, X_valid, y_valid, "Dummy")
)

results.append(evaluate_clf(
    Pipeline([
        ("prep", preprocessor),
        ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
    ]),
    X_train, y_train, X_valid, y_valid, "LogReg (balanced)")
)

results.append(evaluate_clf(
    ImbPipeline([
        ("prep", preprocessor),
        ("smote", SMOTE(random_state=RNG)),
        ("model", LogisticRegression(max_iter=1000))
    ]),
    X_train, y_train, X_valid, y_valid, "LogReg + SMOTE")
)

results.append(evaluate_clf(
    Pipeline([
        ("prep", preprocessor),
        ("model", GradientBoostingClassifier(random_state=RNG))
    ]),
    X_train, y_train, X_valid, y_valid, "GradientBoosting")
)

pd.DataFrame([{k: v for k, v in r.items() if k != "pipeline"} for r in results])

In [ ]:
# Pick best by ROC-AUC
best = sorted(results, key=lambda d: d["roc_auc"], reverse=True)[0]
best_name = best["name"]
best_pipe = best["pipeline"]
best_name

In [ ]:
# Hyperparameter tuning on GradientBoosting
param_dist = {
    "model__n_estimators": [150, 200, 300],
    "model__learning_rate": [0.05, 0.075, 0.1],
    "model__max_depth": [2, 3, 4],
}
search = RandomizedSearchCV(
    Pipeline([
        ("prep", preprocessor),
        ("model", GradientBoostingClassifier(random_state=RNG))
    ]),
    param_distributions=param_dist,
    n_iter=8,
    cv=3,
    scoring="roc_auc",
    random_state=RNG,
    n_jobs=-1,
)
search.fit(X_train, y_train)
{'best_params': search.best_params_, 'best_cv_roc_auc': search.best_score_}

In [ ]:
# Refit tuned model on train+valid, evaluate on test
tuned_pipe = search.best_estimator_
tuned_pipe.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

proba_test = tuned_pipe.predict_proba(X_test)[:,1]
preds_test = (proba_test >= 0.5).astype(int)
roc_test = roc_auc_score(y_test, proba_test)
pr_test = average_precision_score(y_test, proba_test)
cm = confusion_matrix(y_test, preds_test)

report = classification_report(y_test, preds_test, target_names=["<=50K", ">50K"], output_dict=True)
roc_test, pr_test, cm, report

In [ ]:
# ROC and PR curves
fpr, tpr, _ = roc_curve(y_test, proba_test)
prec, rec, _ = precision_recall_curve(y_test, proba_test)

fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].plot(fpr, tpr, label=f"ROC-AUC={roc_test:.3f}")
ax[0].plot([0,1],[0,1],'--', color='gray')
ax[0].set_title("ROC")
ax[0].set_xlabel("FPR")
ax[0].set_ylabel("TPR")

ax[1].plot(rec, prec, label=f"PR-AUC={pr_test:.3f}")
ax[1].set_title("Precision-Recall")
ax[1].set_xlabel("Recall")
ax[1].set_ylabel("Precision")
plt.tight_layout()
curves_fig = FIG_DIR / "roc_pr_curves.png"
plt.savefig(curves_fig)
curves_fig

In [ ]:
# Confusion matrix
disp = ConfusionMatrixDisplay(cm, display_labels=["<=50K", ">50K"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix (test)")
cm_fig = FIG_DIR / "confusion_matrix.png"
plt.savefig(cm_fig)
cm_fig

In [ ]:
# SHAP summary (on a sample for speed)
X_sample = X_test.sample(n=min(400, len(X_test)), random_state=RNG)
X_proc = tuned_pipe.named_steps["prep"].transform(X_sample)
if hasattr(X_proc, "toarray"):
    X_proc = X_proc.toarray()
feature_names = tuned_pipe.named_steps["prep"].get_feature_names_out()
explainer = shap.Explainer(tuned_pipe.named_steps["model"], X_proc, feature_names=feature_names)
shap_values = explainer(X_proc)
shap.plots.beeswarm(shap_values, max_display=15, show=False)
plt.tight_layout()
shap_fig = FIG_DIR / "shap_beeswarm.png"
plt.savefig(shap_fig, bbox_inches="tight")
shap_fig

In [ ]:
# Slice metrics (by sex and marital.status)
slice_cols = ["sex", "marital.status"]
slice_metrics = []
df_test = X_test.copy()
df_test["y"] = y_test.values
df_test["proba"] = proba_test
df_test["pred"] = preds_test
for col in slice_cols:
    for level, grp in df_test.groupby(col):
        roc = roc_auc_score(grp["y"], grp["proba"]) if grp["y"].nunique()>1 else np.nan
        pr = average_precision_score(grp["y"], grp["proba"]) if grp["y"].nunique()>1 else np.nan
        slice_metrics.append({"slice": f"{col}={level}", "roc_auc": roc, "avg_precision": pr, "n": len(grp)})
pd.DataFrame(slice_metrics).sort_values("roc_auc", ascending=False).head(10)

## Assess (summary)
- Best model: tuned GradientBoostingClassifier with one-hot + scaling, evaluated on test.
- Report: see classification_report, ROC/PR curves, confusion matrix, slices, and SHAP.
- Deployment: bundle preprocess + model, monitor slice metrics and calibration.